## Building GP Models from Scratch
Sometimes it is useful to build GP models outside the context of BO for data
visualization and senativity measurements, ie. learned hyperparameters. Here we
demonstrate how to build models from data outside of generators.

For this we use the 3D rosenbrock function test function.

In [ ]:
# set values if testing
import os

from xopt import Xopt, Evaluator
from xopt.generators import RandomGenerator
from xopt.resources.test_functions.rosenbrock import (
    evaluate_rosenbrock,
    make_rosenbrock_vocs,
)

from xopt.generators.bayesian.visualize import visualize_model
from xopt.generators.bayesian.models.standard import StandardModelConstructor

# Ignore all warnings
import warnings

warnings.filterwarnings("ignore")


SMOKE_TEST = os.environ.get("SMOKE_TEST")
NUM_MC_SAMPLES = 1 if SMOKE_TEST else 128
NUM_RESTARTS = 1 if SMOKE_TEST else 20

# make rosenbrock function vocs in 3D
vocs = make_rosenbrock_vocs(3)

# collect some data using random sampling
evaluator = Evaluator(function=evaluate_rosenbrock)
generator = RandomGenerator(vocs=vocs)
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X.random_evaluate(15)

## Create GP model based on the data

In [ ]:
data = X.data

In [ ]:
model_constructor = StandardModelConstructor()

# here we build a model from vocs
model = model_constructor.build_model_from_vocs(vocs=vocs, data=data)

# here we build a model from info (more flexible)
model = model_constructor.build_model(
    input_names=["x0", "x1", "x2"], outcome_names=["y"], data=data
)
help(model_constructor.build_model)

## Examine GP model hyperparameters
Here we look at the GP hyperparameters for the objective function (the first model).
Note: the hyperparameters here are in raw_units (due to contraints on parameter
values, ie. lengthscales > 0)

In [ ]:
objective_model = model.models[vocs.output_names.index("y")]

# print raw hyperparameter values
for name, val in objective_model.named_parameters():
    print(name, val)

In [ ]:
# print real values - note that these are in normalized coordinates
print("outputscale:", objective_model.covar_module.outputscale.data)
print("prior mean:", objective_model.mean_module.constant.data)
print("noise:", objective_model.likelihood.noise_covar.noise.data)
print("lengthscales", objective_model.covar_module.base_kernel.lengthscale.data)

## Visualize model predictions

In [ ]:
fig, ax = visualize_model(
    model, vocs, data, variable_names=["x0", "x1"], reference_point={"x2": 0.0}
)